# IMPORT LIBRARY

In [1]:
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.metrics import f1_score


# DATA LOAD

In [2]:
# 파일 불러오기
file_path = '../train_data//merge_train_preprocess_refine_columns.csv'
data = pd.read_csv(file_path)

In [23]:
data.head()

,SUBCLASS,ID,BRCA1,HMGB3,PLXNB2,NOTCH2,RYR2,ALB,BTG1,SMC1A,...,PEX6,B2M,MYLK,BTG2,CACNA1B,PABPC1,FGFR3,CLIP2,CDH1,HELZ2
0,KIPAN,TRAIN_0000,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,SARC,TRAIN_0001,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,A1690V
2,SKCM,TRAIN_0002,WT,WT,WT,P2219S,WT,WT,WT,WT,...,R343W,WT,P948P,WT,WT,WT,WT,WT,WT,WT
3,KIRC,TRAIN_0003,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,GBMLGG,TRAIN_0004,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


# 데이터 샘플링

In [3]:
# 목표 데이터 수 설정
target_count = 200

# 각 subclass별로 200개의 데이터를 추출하고, 부족한 경우 복제해서 채움
balanced_data = data.groupby('SUBCLASS', group_keys=False).apply(
    lambda x: x.sample(n=target_count, replace=True, random_state=1)
)

# 결과 확인
print(balanced_data['SUBCLASS'].value_counts())

# 데이터 저장
balanced_data.to_csv('../train_data/balanced_data_200.csv', index=False)


ValueError: Cannot take a larger sample than population when 'replace=False'

In [5]:
train = balanced_data.copy()

# 인코딩

In [7]:
# SUBCLASS 가 범주형이기 때문에 LabelEncoder 사용
le_subclass = LabelEncoder()
train['SUBCLASS'] = le_subclass.fit_transform(train['SUBCLASS'])

# 변환된 레이블 확인
for i, label in enumerate(le_subclass.classes_):
    print(f"원래 레이블: {label}, 변환된 숫자: {i}")

원래 레이블: ACC, 변환된 숫자: 0
원래 레이블: BLCA, 변환된 숫자: 1
원래 레이블: BRCA, 변환된 숫자: 2
원래 레이블: CESC, 변환된 숫자: 3
원래 레이블: COAD, 변환된 숫자: 4
원래 레이블: DLBC, 변환된 숫자: 5
원래 레이블: GBMLGG, 변환된 숫자: 6
원래 레이블: HNSC, 변환된 숫자: 7
원래 레이블: KIPAN, 변환된 숫자: 8
원래 레이블: KIRC, 변환된 숫자: 9
원래 레이블: LAML, 변환된 숫자: 10
원래 레이블: LGG, 변환된 숫자: 11
원래 레이블: LIHC, 변환된 숫자: 12
원래 레이블: LUAD, 변환된 숫자: 13
원래 레이블: LUSC, 변환된 숫자: 14
원래 레이블: OV, 변환된 숫자: 15
원래 레이블: PAAD, 변환된 숫자: 16
원래 레이블: PCPG, 변환된 숫자: 17
원래 레이블: PRAD, 변환된 숫자: 18
원래 레이블: SARC, 변환된 숫자: 19
원래 레이블: SKCM, 변환된 숫자: 20
원래 레이블: STES, 변환된 숫자: 21
원래 레이블: TGCT, 변환된 숫자: 22
원래 레이블: THCA, 변환된 숫자: 23
원래 레이블: THYM, 변환된 숫자: 24
원래 레이블: UCEC, 변환된 숫자: 25


In [8]:
## x 의 경우도 범주형으로 구성되어 있어, 알맞은 인코딩 필요
X = train.drop(columns=['SUBCLASS', 'ID'])
y_subclass = train['SUBCLASS']

categorical_columns = X.select_dtypes(include=['object', 'category']).columns
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_encoded = X.copy()
X_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

# 데이터 분할

In [9]:
# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_subclass, test_size=0.1, stratify=y_subclass, random_state=42)


# 모델 학습

In [10]:
model = xgb.XGBClassifier(
    n_estimators=600,
    learning_rate=0.1,
    max_depth=12,
    colsample_bytree = 0.7,
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss'
)

In [11]:
model.fit(X_train, y_train)

c:\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:21:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=12,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=600,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [12]:
# 모델 평가
y_pred = model.predict(X_test)
accuracy = (y_pred == y_test).mean()
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.54


# 추론

In [13]:
col = list(X_train.columns)

In [15]:
len(col)

148

In [16]:
test = pd.read_csv('../train_data/test_preprocessing.csv')

In [17]:
test_X = test[col]
X_encoded = test_X.copy()
X_encoded[categorical_columns] = ordinal_encoder.transform(test_X[categorical_columns])

In [18]:
predictions = model.predict(X_encoded)

In [19]:
original_labels = le_subclass.inverse_transform(predictions)

In [20]:
submisson = pd.read_csv("../train_data/sample_submission.csv")

In [21]:
submisson["SUBCLASS"] = original_labels

In [22]:
submisson.to_csv('../submission_data/xgb_subclass_sampling.csv', encoding='UTF-8-sig', index=False)